In [2]:

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/home/casapanshop/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


614400/600901 [==============================] - 1s 2us/step
corpus length: 600893
total chars: 57
nb sequences: 200285


In [8]:
print(sentences[:3])
#print(next_chars[0])

preface


supposing that truth is a woma
n


In [4]:
print('Vectorization... One Hot Vector')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print(x.shape, y.shape)

Vectorization... One Hot Vector
(200285, 40, 57) (200285, 57)


In [ ]:
##mejoras:
-embedding para characteres
-tfidf para caracteres.

In [5]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)),return_sequences=False,stateful=False))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
_________________________________________________________________
activation_1 (Activation)    (None, 57)                0         
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [ ]:
##mejoras:
otra LSTM entemedio
una Dense entremedio
una GRU envez
con stateful True, shuffle=False


##variantes:
cantidad de caracteres como vector

In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence): #input
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char #for next character

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,batch_size=256,epochs=60, callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 72s 358us/step - loss: 1.5486

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "all interpreters, who lifted the entire "
all interpreters, who lifted the entire of the prospection of the such a man is the supernorable to the soul, the sacrifice of the senter to the self-derection of the proporations of the prospection of the soul, the self-desire of the serves of the prospection of the soul, the subjection of the prospection of the prospice to the prospection of the soul, the serves and the experient of the fact the soul, the such as the serves of the sel
----- diversity: 0.5
----- Generating with seed: "all interpreters, who lifted the entire "
all interpreters, who lifted the entire and without the soulis of the german every conscience, that and way despines and contempt of consequently in the saccasion of the sacrificistic of the instance. the power of the problemstic so the word the p

the should by long are supiocally this, thereby:
a bad
capearth
or
again
sentency and idesposed when their highest honeer as man of the
very divider murtered
bettem of--erron of which be reasol, in their child weres is as it itself, ot has symptoms. from all thought is nights as as retertary to them, diselven
to preature of day 
----- diversity: 1.2
----- Generating with seed: " creating
is a law-giving, their will to"
 creating
is a law-giving, their will to lived, whom they be selfeious, think to saym we are kneatn woathom-aborth, whether
apads
danger is buct, is out and were not us,
injorable ourselves; it
loled enolush, is ntad
alse knowled: then" furkeron woute is, who rither, "kpeopds, to , wim tighten
hitherselffnceoud bendfheend
actuane of sifflered; upon is the
power: it. but for him.--are proud
beneedles itself of a metaphysial
cintle
the re
Epoch 5/60
200285/200285 [==============================] - 73s 367us/step - loss: 1.3935

----- Generating text after Epoch: 4
----- di

disadvantage, attending the termination of the sentiments of the sentiments and and all the sentiments and as a philosophers and the sentiments and man and in the seriest and called by the sentiments of the present and the seriest of the sentiments and conscience and a soul, and the strength and the sense and all the serves and sentiments to the serves and and all the personally the sentiments and belief in the sentiments and the sens
----- diversity: 0.5
----- Generating with seed: "d
disadvantage, attending the terminatio"
d
disadvantage, attending the termination of the power the everything.=--this person of the powerful of the spirit to the existence of the more and everything the expleases and soul. in the place to the are as a person of their realled and providicing and a proper and the belief of retertions of the relations to really refined the very and a conscience the considerable as a live as the present and their and sellous to the laws and every
----- diversity: 1.0
----- G

spiritualen ebimitia stands self will. so man people, developed perhaps, origits add not be does nomeling general unself, tend wholy or taste. freedems." let shas
standsess sandary, the innuirs. 
----- diversity: 1.2
----- Generating with seed: "ry men feel
exactly the opposite: the ri"
ry men feel
exactly the opposite: the rigression of night of he cirplonis
self mastounity!

"c gable," so, where to many lawss
dimbehour sile;
todard and "honouring
out of kind oh inflictic toon sumpulity. and penuality: banajd to fon a woventarce a
meny easter
nearilesing esseltify it immediator
the unside or capacity that idea"zle,--their proud of livtl and coill'ssur, conseive of the made gaines.=.=--emotionty, life heavourred to pre
Epoch 12/60
200285/200285 [==============================] - 72s 361us/step - loss: 1.3204

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: ", and the
like; therefore, because the w"
, and the
like; therefore, because the will to th

KeyboardInterrupt: 

### Predict on test time.

In [32]:

generated = ''
sentence = "as a mirror, that the deep heavens may !" #rellenar con un simbolo de "null" #
generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

diversity=1
for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence): #input
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity) #not deterministic
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char #for next character

    sys.stdout.write(next_char)
    sys.stdout.flush()

----- Generating with seed: "as a mirror, that the deep heavens may !"
as a mirror, that the deep heavens may ! somein, why the
judgment and exist, and for will was found consideration for exalted that is a churhce, science, what or tejorbomen self-danghe himself for purfidentable germans in the fear of the most mit down by all ourselves to the estimate of his more for his it, here--as a new and bad given. knows to indeed and heart, in beott and of
suffering one in the genius of common, is characters of
re